In [70]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, FunctionTransformer

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

In [71]:
# Directory config to project root to insure consistency across environments for project specific imports
from pyprojroot import here
os.chdir(here())

# Project specific imports
from src.utils import save_dataframe


## Data Quering and Loading

In [72]:
# DB related imports
from database.db_utils import init_db
from config.config_loader import load_config
from database.queries import prepped_data_query
from sqlalchemy import text

# Initialize local PostgreSQL session
Session = init_db(load_config("DB_URL"))
session = Session()

# Execute and fetch results
data = session.execute(text(prepped_data_query))

# Close the session
session.close()

# Convert to DataFrame
data = pd.DataFrame(data.fetchall(), columns=data.keys())

In [73]:
# Saving cleaned dataset for future use
save_dataframe(data, "00_base_data.csv")

✅ Data successfully saved to ./data\00_base_data.csv with separator ','


In [74]:
# Load dataset
data = pd.read_csv("./data/00_base_data.csv")

## Initial Data Overview

In [75]:
# data["missing_budget"] = (data["budget"] == 0).astype(int)
# data["missing_revenue"] = (data["revenue"] == 0).astype(int)

In [76]:
# data["budget"] = data["budget"].replace(0, np.nan)
# data["revenue"] = data["revenue"].replace(0, np.nan)

In [77]:
# data = data[(data["budget"] != 0) & (data["revenue"] != 0)]

In [78]:
# data = data[(data["budget"] != 0) | (data["revenue"] != 0)]

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5997 entries, 0 to 5996
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    5997 non-null   object 
 1   release_date             5997 non-null   object 
 2   tmdb_vote_count          5997 non-null   int64  
 3   tmdb_vote_average        5997 non-null   float64
 4   genre_names              5997 non-null   object 
 5   budget                   3734 non-null   float64
 6   revenue                  4160 non-null   float64
 7   runtime_in_min           5995 non-null   float64
 8   tmdb_popularity          5997 non-null   float64
 9   production_company_name  5971 non-null   object 
 10  production_country_name  5992 non-null   object 
 11  spoken_languages         5994 non-null   object 
 12  director                 5994 non-null   object 
 13  writer                   5941 non-null   object 
 14  actors                  

In [80]:
# Count missing values
data.isnull().sum()

title                         0
release_date                  0
tmdb_vote_count               0
tmdb_vote_average             0
genre_names                   0
budget                     2263
revenue                    1837
runtime_in_min                2
tmdb_popularity               0
production_company_name      26
production_country_name       5
spoken_languages              3
director                      3
writer                       56
actors                        8
imdb_rating                   9
imdb_votes                    7
metascore                  1489
age_rating                  435
awards                     1003
rotten_tomatoes_rating      928
meta_critic_rating         1489
dtype: int64

In [81]:
data.head(5)

,title,release_date,tmdb_vote_count,tmdb_vote_average,genre_names,budget,revenue,runtime_in_min,tmdb_popularity,production_company_name,...,director,writer,actors,imdb_rating,imdb_votes,metascore,age_rating,awards,rotten_tomatoes_rating,meta_critic_rating
0,Carter,2022-08-05,378,6.000,"Action, Thriller, Crime",NaN,NaN,132.0,27.043,Apeitda,...,Jung Byung-gil,"Jung Byung-gil, Byeong-sik Jung","Joo Won, Kim Bo-min, Lee Sung-jae",5.1,"10,297",NaN,TV-MA,NaN,32.0,NaN
1,The Shadow in My Eye,2021-10-28,494,7.829,"War, Drama, History",NaN,NaN,107.0,16.008,Miso Film,...,Ole Bornedal,Ole Bornedal,"Bertram Bisgaard Enevoldsen, Ester Birch, Ella...",7.3,"17,277",NaN,TV-MA,2 wins & 13 nominations,NaN,NaN
2,Noelle,2019-11-12,673,6.595,"Family, Comedy, Fantasy",NaN,NaN,100.0,10.040,Walt Disney Pictures,...,Marc Lawrence,Marc Lawrence,"Anna Kendrick, Shirley MacLaine, Bill Hader",6.3,"28,616",48.0,G,1 win total,56.0,48.0
3,Play,2019-09-04,349,7.278,Comedy,NaN,NaN,108.0,5.214,"Chapter 2, Moonshaker",...,Anthony Marciano,"Max Boublil, Anthony Marciano","Max Boublil, Alice Isaaz, Malik Zidi",7.2,"1,755",NaN,NaN,1 nomination,NaN,NaN
4,Dave Chappelle: Sticks & Stones,2019-08-26,320,7.600,Comedy,NaN,NaN,65.0,7.683,NaN,...,Stan Lathan,Dave Chappelle,Dave Chappelle,8.3,"28,457",NaN,TV-MA,Won 3 Primetime Emmys. 4 wins & 5 nominations ...,42.0,NaN


## Multilable Categorical Features

There are a nuymber of multilable categorical features that will need to be looked into.

In [82]:
def count_unique_values_for_feature(df: pd.DataFrame, feature: str, delimiter: str = ",") -> int:
    """
    Splits the specified feature column by the delimiter and returns the number of unique values.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        feature (str): The name of the column to process.
        delimiter (str): The delimiter used to separate multiple values in the column.

    Returns:
        int: The number of unique values.
    """
    return len(df[feature].dropna().str.split(rf"{delimiter}\s*").explode().unique())

# List of features you want to analyze:
features = [
    "genre_names", 
    "production_company_name", "production_country_name", 
            "spoken_languages", "director", "writer", "actors"]

# Create a dictionary with the counts for each feature:
unique_counts = {feature: count_unique_values_for_feature(data, feature) for feature in features}

# Display the results:
for feature, count in unique_counts.items():
    print(f"{feature}: {count} unique values")

genre_names: 19 unique values
production_company_name: 7238 unique values
production_country_name: 96 unique values
spoken_languages: 100 unique values
director: 3445 unique values
writer: 7741 unique values
actors: 7822 unique values


In [83]:
def print_top_categories(df: pd.DataFrame, column: str, top_n: int, delimiter: str = ",", others_label: str = "Others") -> None:
    """
    Prints the top_n unique values from a multi-label column and the total count of values 
    that fall outside the top_n (which would be grouped as 'Others').

    Args:
        df (pd.DataFrame): The DataFrame containing your data.
        column (str): The name of the multi-label column.
        top_n (int): The number of top categories to display.
        delimiter (str): The delimiter separating multiple values (default is a comma).
        others_label (str): The label used for less frequent values.
    """
    # Split the column into individual values and count frequencies
    exploded = df[column].dropna().str.split(rf"{delimiter}\s*").explode().str.strip()
    counts = exploded.value_counts()
    
    # Get the top N categories and the sum for the rest
    top_categories = counts.head(top_n)
    others_count = counts[counts.index.difference(top_categories.index)].sum()
    
    print("--------------------------------------------------||")
    print(f"Top {top_n} unique values for '{column}':")
    print(top_categories)
    print(f"Total count of all other values (will be grouped as '{others_label}'): {others_count}")
    print("--------------------------------------------------||\n")


top_values = {
    "genre_names": 20,
    "production_company_name": 20,
    "production_country_name": 10,
    "spoken_languages": 10,
    "director": 20,
    "writer": 20,
    "actors": 20
}

for feature, top_n in top_values.items():
    print_top_categories(data, feature, top_n)

--------------------------------------------------||
Top 20 unique values for 'genre_names':
genre_names
Drama              2618
Comedy             2053
Thriller           1690
Action             1431
Adventure           960
Horror              938
Romance             907
Crime               797
Science Fiction     743
Fantasy             675
Family              642
Animation           591
Mystery             561
History             311
Music               173
War                 158
Documentary         117
TV Movie             87
Western              57
Name: count, dtype: int64
Total count of all other values (will be grouped as 'Others'): 0
--------------------------------------------------||

--------------------------------------------------||
Top 20 unique values for 'production_company_name':
production_company_name
Universal Pictures       250
Warner Bros. Pictures    215
Columbia Pictures        187
Lionsgate                183
Paramount Pictures       151
20th Century Fox    

In [84]:
def group_top_categories(df: pd.DataFrame, column: str, top_n: int, delimiter: str = ",", others_label: str = "Others") -> pd.Series:
    """
    Processes a multi-label column by keeping only the top_n categories (based on frequency) 
    and replacing any other category with the 'others_label'.

    Args:
        df (pd.DataFrame): The DataFrame containing your data.
        column (str): The name of the multi-label column.
        top_n (int): The number of top categories to keep.
        delimiter (str): The delimiter separating multiple values in the column.
        others_label (str): The label to use for all categories not in the top_n.

    Returns:
        pd.Series: A new Series with the modified values.
    """
    # Split the column and count frequencies
    exploded = df[column].dropna().str.split(rf"{delimiter}\s*").explode().str.strip()
    counts = exploded.value_counts()
    top_categories = counts.head(top_n).index.tolist()
    
    def map_categories(cell):
        if pd.isna(cell):
            return cell
        cats = [cat.strip() for cat in cell.split(delimiter)]
        # Replace any category not in top_categories with others_label
        new_cats = [cat if cat in top_categories else others_label for cat in cats]
        # Remove duplicates while preserving order
        seen = set()
        new_cats = [x for x in new_cats if x not in seen and not seen.add(x)]
        return delimiter.join(new_cats)
    
    return df[column].apply(map_categories)

top_values = {
    # "genre_names": 20,  # Keeping all genres
    # "production_company_name": 20,  # Too granular
    "production_country_name": 5,
    "spoken_languages": 5,
    # "director": 20,  # Too granular
    # "writer": 20,  # Too granular
    # "actors": 20  # Too granular
}

# for feature, top_n in top_values.items():
#     data[feature] = group_top_categories(data, feature, top_n)

# data.head()


## Handling Nulls

### Simple Imputer

#### Categorical Imputer

In [85]:
from sklearn.compose import make_column_selector

num_cols = make_column_selector(dtype_include=['number'])
num_cols = num_cols(data)
print(num_cols)

['tmdb_vote_count', 'tmdb_vote_average', 'budget', 'revenue', 'runtime_in_min', 'tmdb_popularity', 'imdb_rating', 'metascore', 'rotten_tomatoes_rating', 'meta_critic_rating']


In [86]:
cat_cols = make_column_selector(dtype_include=['object'])
cat_cols = cat_cols(data)
print(cat_cols)

['title', 'release_date', 'genre_names', 'production_company_name', 'production_country_name', 'spoken_languages', 'director', 'writer', 'actors', 'imdb_votes', 'age_rating', 'awards']


In [87]:
# Imputer cols
cat_cols = ['production_company_name', 'production_country_name', 'spoken_languages', 'director', 'writer', 'actors', 'age_rating']
zero_cols = ['imdb_rating', 'imdb_votes']
median_cols = ['runtime_in_min']
columns_to_drop = [
    # 'title', 
    # 'awards',
    'release_date'
    ]

In [88]:
# Iter columns setup
iter_cols = [
    'metascore', 'rotten_tomatoes_rating', 'meta_critic_rating',
    'budget', 'revenue'
    ]

predictor_cols = [
    'tmdb_vote_count', 'tmdb_vote_average', 'runtime_in_min', 'tmdb_popularity', 'imdb_rating', 
    'imdb_votes', 'total_wins', 'total_noms', 'oscar_wins', 'oscar_noms', 'bafta_wins', 'bafta_noms',
    'metascore_missing', 'rotten_tomatoes_rating_missing', 'meta_critic_rating_missing',
    ]

full_iter_cols = iter_cols + predictor_cols

In [89]:
# Categorical columns imputer
cat_imputer = SimpleImputer(strategy="constant", fill_value="Unknown")

cat_pipeline = Pipeline(steps=[
    ('cat_imputer', cat_imputer)
    # ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Numerical to zero columns imputer
zero_imputer = SimpleImputer(strategy='constant', fill_value=0)

zero_pipeline = Pipeline(steps=[
    ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0))
    # ('scaler', StandardScaler())
])


# Numerical to median columns imputer
median_imputer = SimpleImputer(strategy='median')

median_pipeline = Pipeline(steps=[
    ('median_imputer', median_imputer)
    # ('scaler', StandardScaler())
])


# Iterative imputer
iter_imputer = IterativeImputer(
    estimator=BayesianRidge(),
    # estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    max_iter=10,
    initial_strategy='median',
    n_nearest_features=None,
    imputation_order='ascending',
    random_state=42
    )

iter_pipeline = Pipeline(steps=[
    ('cat_imputer', iter_imputer)
    # ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

### Custom Transformers

In [90]:
# Define a function to add missing indicators for certain columns.
def add_missing_indicators(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        df[col + "_missing"] = df[col].isnull().astype(int)
    return df

missing_indicator_transformer = FunctionTransformer(add_missing_indicators, validate=False)


# Define a function transformer to drop unwanted columns.
def drop_unwanted_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(columns=columns_to_drop, errors='ignore')

dropper = FunctionTransformer(drop_unwanted_columns)


# To numeric
def convert_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        # Convert to string, remove commas, then convert to numeric
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
    return df

to_numeric = FunctionTransformer(convert_to_numeric, validate=False)


# def convert_to_numeric_cols(df: pd.DataFrame, cols: list) -> pd.DataFrame:
#     df = df.copy()
#     for col in cols:
#         # Remove commas and convert to numeric (coercing errors to NaN)
#         df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
#     return df

# to_numeric_transformer = FunctionTransformer(lambda X: convert_to_numeric_cols(X, cols=['imdb_votes']),
#                                              validate=False)


In [91]:
def extract_awards_info(awards_str):
    """
    Extracts numerical awards information from a text string.

    Parameters
    ----------
    awards_str : str
        The awards description string.

    Returns
    -------
    pd.Series
        A Series with the following index:
        ["total_wins", "total_noms", "oscar_wins", "oscar_noms", "bafta_wins", "bafta_noms"]
    """
    # Handle missing or "N/A" values.
    if pd.isna(awards_str) or awards_str.strip() in ["N/A", ""]:
        return pd.Series([0, 0, 0, 0, 0, 0],
                         index=["total_wins", "total_noms", "oscar_wins", "oscar_noms", "bafta_wins", "bafta_noms"])
    
    # Extract overall totals.
    # Look for a pattern like "56 wins" (we use negative lookahead to avoid picking up Oscar wins)
    total_wins_match = re.search(r'(\d+)\s+wins?(?!.*Oscars)', awards_str, flags=re.IGNORECASE)
    total_noms_match = re.search(r'(\d+)\s+nominations', awards_str, flags=re.IGNORECASE)
    total_wins = int(total_wins_match.group(1)) if total_wins_match else 0
    total_noms = int(total_noms_match.group(1)) if total_noms_match else 0

    # Oscar-specific extraction:
    oscar_noms_match = re.search(r'Nominated for\s+(\d+)\s+Oscars?', awards_str, flags=re.IGNORECASE)
    oscar_noms = int(oscar_noms_match.group(1)) if oscar_noms_match else 0
    # Look for something like "Oscars. 56 wins" or "Oscars 56 wins" (using non-digit separator)
    oscar_wins_match = re.search(r'Oscars?[\W_]+(\d+)\s+wins?', awards_str, flags=re.IGNORECASE)
    oscar_wins = int(oscar_wins_match.group(1)) if oscar_wins_match else 0

    # BAFTA-specific extraction:
    # For nominations, sometimes the text might run together (e.g. "BAFTA Award28 nominations total")
    bafta_noms_match = re.search(r'Nominated for\s+(\d+)\s*BAFTA', awards_str, flags=re.IGNORECASE)
    bafta_noms = int(bafta_noms_match.group(1)) if bafta_noms_match else 0
    # For wins, allow an optional "Award" word after BAFTA.
    bafta_wins_match = re.search(r'BAFTA(?:\s+Award)?[\D_]+(\d+)\s+wins?', awards_str, flags=re.IGNORECASE)
    bafta_wins = int(bafta_wins_match.group(1)) if bafta_wins_match else 0

    return pd.Series([total_wins, total_noms, oscar_wins, oscar_noms, bafta_wins, bafta_noms],
                     index=["total_wins", "total_noms", "oscar_wins", "oscar_noms", "bafta_wins", "bafta_noms"])


def transform_awards(X):
    """
    Expects X to be a DataFrame with a single column (e.g., 'awards').
    Applies extract_awards_info row-wise and returns a DataFrame.
    """
    # Apply the function to the first (and only) column
    return X.iloc[:, 0].apply(extract_awards_info)

# Wrap the function in a FunctionTransformer
awards_transformer = FunctionTransformer(transform_awards, validate=False)

In [92]:
from functools import partial

def transform_top_categories(X, column, top_n, delimiter=",", others_label="Others"):
    """
    Transforms a multi-label column by keeping only the top_n categories (based on frequency)
    and replacing all other categories with a generic label.
    
    Parameters:
        X (pd.DataFrame): Input DataFrame.
        column (str): The name of the multi-label column to process.
        top_n (int): Number of top categories to keep.
        delimiter (str): Delimiter separating the values.
        others_label (str): Label to assign to categories not among the top_n.
    
    Returns:
        pd.DataFrame: A DataFrame with one column (the processed column).
    """
    X = X.copy()
    # Split the column values, explode, and count frequencies.
    exploded = X[column].dropna().str.split(rf"{delimiter}\s*").explode().str.strip()
    counts = exploded.value_counts()
    top_categories = counts.head(top_n).index.tolist()
    
    def map_categories(cell):
        if pd.isna(cell):
            return cell
        # Split and strip each value.
        cats = [cat.strip() for cat in cell.split(delimiter)]
        # Replace values not in top_categories with others_label.
        new_cats = [cat if cat in top_categories else others_label for cat in cats]
        # Remove duplicates while preserving order.
        seen = set()
        new_cats = [x for x in new_cats if x not in seen and not seen.add(x)]
        return delimiter.join(new_cats)
    
    X[column] = X[column].apply(map_categories)
    # Return a DataFrame with just the transformed column.
    return X[[column]]

# Now, to create a FunctionTransformer for, say, the 'production_country_name' column with top_n=5:
transformer_prod_country = FunctionTransformer(
    func=partial(transform_top_categories, column="production_country_name", top_n=5, delimiter=",", others_label="Others"),
    validate=False
)

# Similarly, for 'spoken_languages' column with top_n=5:
transformer_spoken_lang = FunctionTransformer(
    func=partial(transform_top_categories, column="spoken_languages", top_n=5, delimiter=",", others_label="Others"),
    validate=False
)

In [93]:
def add_date_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['release_year'] = df['release_date'].dt.year
    df['release_month'] = df['release_date'].dt.month
    df['release_day'] = df['release_date'].dt.day
    df['is_weekend'] = (df['release_date'].dt.weekday >= 4).astype(int)
    df['is_holiday_season'] = df['release_month'].isin([6, 7, 11, 12]).astype(int)
    df['movie_age'] = 2025 - df['release_year']
    return df

# Wrap the function as a transformer
date_features_transformer = FunctionTransformer(add_date_features, validate=False)

In [94]:
def calculate_roi(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['roi'] = (df['revenue'] - df['budget']) / df['budget']
    return df

# Wrap the function as a transformer
roi_transformer = FunctionTransformer(calculate_roi, validate=False)

### Apply Transformations

In [95]:
drop_transformer = ColumnTransformer(
    transformers=[
        ('dropper', 'drop', columns_to_drop)
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)
# Set output to pandas dataframe
drop_transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('dropper', 'drop', ['release_date'])],
                  verbose_feature_names_out=False)

In [96]:
numeric_transformer = ColumnTransformer(
    transformers=[
        ('to_numeric', to_numeric, zero_cols)
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)
# Set output to pandas dataframe
numeric_transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('to_numeric',
                                 FunctionTransformer(func=<function convert_to_numeric at 0x000001F3B8B7FEC0>),
                                 ['imdb_rating', 'imdb_votes'])],
                  verbose_feature_names_out=False)

In [97]:
main_transformer = ColumnTransformer(
    transformers=[
        ('zero', zero_pipeline, zero_cols),
        ('median', median_pipeline, median_cols),
        ('cat', cat_pipeline, cat_cols),
        ('awards', awards_transformer, ['awards']),
        ('date_features', date_features_transformer, ['release_date']),
        # ('roi', roi_transformer, ['budget', 'revenue']),
        ('dropper', dropper, columns_to_drop)
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)

# Set output to pandas dataframe
main_transformer.set_output(transform='pandas')

# Apply the preprocessor to the data
# clean_data = main_transformer.fit_transform(data)
# clean_data.head()

ColumnTransformer(remainder='passthrough',
                  transformers=[('zero',
                                 Pipeline(steps=[('zero_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['imdb_rating', 'imdb_votes']),
                                ('median',
                                 Pipeline(steps=[('median_imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['runtime_in_min']),
                                ('cat',
                                 Pipeline(steps=[('cat_imputer',
                                                  SimpleImputer(fill_value='Unknown',...
                                  'age_rating']),
                                ('awards',
                                 FunctionTransformer(func=<function transform_awards at 0x000001F20601C040>),
                                 ['awards']),
                                ('date_features',
                                 FunctionTransformer(func=<function add_date_features at 0x000001F205D13F60>),
                                 ['release_date']),
                                ('dropper',
                                 FunctionTransformer(func=<function drop_unwanted_columns at 0x000001F3B8B7FB00>),
                                 ['release_date'])],
                  verbose_feature_names_out=False)

In [98]:
top_n_transformer = ColumnTransformer(
    transformers=[
        ('prod_country', transformer_prod_country, ['production_country_name']),
        ('spoken_lang', transformer_spoken_lang, ['spoken_languages'])
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)

# Set output to pandas dataframe
top_n_transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('prod_country',
                                 FunctionTransformer(func=functools.partial(<function transform_top_categories at 0x000001F205D13880>, column='production_country_name', top_n=5, delimiter=',', others_label='Others')),
                                 ['production_country_name']),
                                ('spoken_lang',
                                 FunctionTransformer(func=functools.partial(<function transform_top_categories at 0x000001F205D13880>, column='spoken_languages', top_n=5, delimiter=',', others_label='Others')),
                                 ['spoken_languages'])],
                  verbose_feature_names_out=False)

In [99]:
missing_indicator_transformer = ColumnTransformer(
    transformers=[
        ('missing_indicator', missing_indicator_transformer, iter_cols),
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)

# Set output to pandas dataframe
missing_indicator_transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('missing_indicator',
                                 FunctionTransformer(func=<function add_missing_indicators at 0x000001F205D13560>),
                                 ['metascore', 'rotten_tomatoes_rating',
                                  'meta_critic_rating', 'budget', 'revenue'])],
                  verbose_feature_names_out=False)

In [100]:
iter_transformer = ColumnTransformer(
    transformers=[
        # ('iter', iter_pipeline, iter_cols),
        ('iter', iter_pipeline, full_iter_cols),
    ],
    remainder='passthrough', 
    verbose_feature_names_out=False
)

# Set output to pandas dataframe
iter_transformer.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('iter',
                                 Pipeline(steps=[('cat_imputer',
                                                  IterativeImputer(estimator=BayesianRidge(),
                                                                   initial_strategy='median',
                                                                   random_state=42))]),
                                 ['metascore', 'rotten_tomatoes_rating',
                                  'meta_critic_rating', 'budget', 'revenue',
                                  'tmdb_vote_count', 'tmdb_vote_average',
                                  'runtime_in_min', 'tmdb_popularity',
                                  'imdb_rating', 'imdb_votes', 'total_wins',
                                  'total_noms', 'oscar_wins', 'oscar_noms',
                                  'bafta_wins', 'bafta_noms',
                                  'metascore_missing',
                                  'rotten_tomatoes_rating_missing',
                                  'meta_critic_rating_missing'])],
                  verbose_feature_names_out=False)

In [101]:
full_pipeline = Pipeline(steps=[
    ('numeric_transformer', numeric_transformer),
    ('main_transformer', main_transformer),
    ('top_n_transformer', top_n_transformer),
    ('missing_indicator_transformer', missing_indicator_transformer),
    ('iter_preprocessor', iter_transformer),
    ('roi', roi_transformer),
    ('date_feature_engineering', date_features_transformer),
    ('drop_transformer', drop_transformer)
])

# Set output to pandas dataframe
full_pipeline.set_output(transform='pandas')

clean_data = full_pipeline.fit_transform(data)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5997 entries, 0 to 5996
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   metascore                       5997 non-null   float64
 1   rotten_tomatoes_rating          5997 non-null   float64
 2   meta_critic_rating              5997 non-null   float64
 3   budget                          5997 non-null   float64
 4   revenue                         5997 non-null   float64
 5   tmdb_vote_count                 5997 non-null   float64
 6   tmdb_vote_average               5997 non-null   float64
 7   runtime_in_min                  5997 non-null   float64
 8   tmdb_popularity                 5997 non-null   float64
 9   imdb_rating                     5997 non-null   float64
 10  imdb_votes                      5997 non-null   float64
 11  total_wins                      5997 non-null   float64
 12  total_noms                      59

In [102]:
clean_data.head(20)

,metascore,rotten_tomatoes_rating,meta_critic_rating,budget,revenue,tmdb_vote_count,tmdb_vote_average,runtime_in_min,tmdb_popularity,imdb_rating,...,age_rating,release_year,release_month,release_day,is_weekend,is_holiday_season,movie_age,title,genre_names,roi
0,57.0,32.000000,57.0,1.734276e+07,-1.457323e+07,378.0,6.000,132.0,27.043,5.1,...,TV-MA,2022,8,5,1,0,3,Carter,"Action, Thriller, Crime",-1.840306
1,57.0,71.442247,57.0,2.316073e+07,1.852967e+07,494.0,7.829,107.0,16.008,7.3,...,TV-MA,2021,10,28,0,0,4,The Shadow in My Eye,"War, Drama, History",-0.199953
2,48.0,56.000000,48.0,2.343006e+07,2.006035e+07,673.0,6.595,100.0,10.040,6.3,...,G,2019,11,12,0,1,6,Noelle,"Family, Comedy, Fantasy",-0.143820
3,57.0,68.884678,57.0,2.191555e+07,1.144714e+07,349.0,7.278,108.0,5.214,7.2,...,Unknown,2019,9,4,0,0,6,Play,Comedy,-0.477671
4,57.0,42.000000,57.0,1.848628e+07,-8.069549e+06,320.0,7.600,65.0,7.683,8.3,...,TV-MA,2019,8,26,0,0,6,Dave Chappelle: Sticks & Stones,Comedy,-1.436515
5,57.0,49.629856,57.0,1.969287e+07,-1.199619e+06,332.0,5.337,105.0,6.277,5.2,...,Unknown,2018,8,8,0,0,7,"Neuilly sa mère, sa mère !",Comedy,-1.060916
6,57.0,47.000000,57.0,1.985523e+07,-2.796386e+05,461.0,7.700,79.0,5.702,8.0,...,TV-MA,2018,3,13,0,0,7,Ricky Gervais: Humanity,Comedy,-1.014084
7,78.0,98.000000,78.0,2.078955e+07,5.037923e+06,323.0,7.568,79.0,7.949,7.6,...,Not Rated,2017,1,8,1,0,8,Kedi,Documentary,-0.757670
8,57.0,59.487687,57.0,2.250524e+07,1.480207e+07,469.0,6.100,94.0,8.383,6.1,...,TV-14,2017,4,26,0,0,8,Wedding Unplanned,"Romance, Comedy",-0.342283
9,57.0,63.000000,57.0,2.300081e+07,1.761988e+07,504.0,5.600,89.0,15.406,5.5,...,Not Rated,2015,10,1,0,0,10,Howl,Horror,-0.233945


## Feature Engineering

In [103]:
# clean_data['release_date'] = pd.to_datetime(clean_data['release_date'])
# clean_data['release_year'] = clean_data['release_date'].dt.year
# clean_data['release_month'] = clean_data['release_date'].dt.month
# clean_data['release_day'] = clean_data['release_date'].dt.day

# clean_data['is_weekend'] = (clean_data['release_date'].dt.weekday >= 4).astype(int)  # 1 if Fri-Sun
# clean_data['is_holiday_season'] = clean_data['release_month'].isin([6, 7, 11, 12]).astype(int)  # Summer & winter holidays
# clean_data['movie_age'] = 2025 - clean_data['release_year']  # If predicting in 2025

# clean_data.drop(columns=['release_date'], inplace=True)


In [104]:
# clean_data['roi'] = (clean_data['revenue'] - clean_data['budget']) / clean_data['budget']

In [105]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5997 entries, 0 to 5996
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   metascore                       5997 non-null   float64
 1   rotten_tomatoes_rating          5997 non-null   float64
 2   meta_critic_rating              5997 non-null   float64
 3   budget                          5997 non-null   float64
 4   revenue                         5997 non-null   float64
 5   tmdb_vote_count                 5997 non-null   float64
 6   tmdb_vote_average               5997 non-null   float64
 7   runtime_in_min                  5997 non-null   float64
 8   tmdb_popularity                 5997 non-null   float64
 9   imdb_rating                     5997 non-null   float64
 10  imdb_votes                      5997 non-null   float64
 11  total_wins                      5997 non-null   float64
 12  total_noms                      59

## Save cleanded data

In [106]:
# Saving cleaned dataset for future use
save_dataframe(clean_data, "01_clean_data.csv")

✅ Data successfully saved to ./data\01_clean_data.csv with separator ','
